# Deep Learning for Vision

- Convolution Operation
- Pooling
- Convolutional Neural Networks



## Convolution Operation





## Convolutional Neural Networks (CNNs)


Neural Networks vs Convolutional Neural Networks
In a typical neural network, each neuron in the input layer is connected to a neuron in the hidden layer. However, in a CNN, only a small region of input layer neurons connect to neurons in the hidden layer. These regions are referred to as local receptive fields, which is translated across an image to create a feature map from the input layer to the hidden layer neurons. To implement this process efficiently, convolution is applied.

In CNNs, the weights and bias values are the same for all hidden neurons in a given layer. This means that all hidden neurons are detecting the same feature such as an edge or a blob in different regions of the image. This makes the network tolerant to translation of objects in an image. For example, a network trained to recognize cats, will be able to do so, whenever the cat is in the image.

### Activation and Pooling
The activation step applies a transformation the the output of the neuron by using activation functions. Rectified Linear Unit, or ReLU, is a commonly used activation function. It takes the output of a neuron and maps it to the highest positive value. If the output is negative, the function maps it to zero.

Pooling further transforms the output of the activation step and reduces the dimensionality of the featured map by condensing the output of small regions of neurons into a single output, which helps simplifying the following layers, and reduces the number of parameters that the model needs to learn.

Every hidden layer increases the complexity of the learned image features. For example, the first layer might learn how to detect edges, and the last layer might learn how to detect more complex shapes.

### Backpropagation
Short for "backward propagation of errors", it is a supervised algorithm used to minimize errors in predictions made by neural networks. The chain rule in calculus is applied: 
$$\frac{dx}{du}=\frac{du}{dy}\times \frac{dx}{dv}$$

If $y=g(u)$ and $u=f(x)$ then $y=g(f(x))$